In [0]:
!pip install -r requirements.txt

In [0]:
from langchain.tools import tool
import json
import pandas as pd
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from databricks_langchain import ChatDatabricks
import os
import mlflow
import matplotlib.pyplot as plt
from datetime import timedelta
import io
import base64
from dotenv import load_dotenv

load_dotenv()

mlflow.langchain.autolog()

In [0]:
llm = ChatDatabricks(model='databricks-llama-4-maverick', temperature=0)

In [0]:
@tool
def consultar_metricas_srag() -> str:
    '''Consulta métricas pré-calculadas, gera gráficos de casos recentes e retorna tudo em JSON (incluindo base64 dos gráficos).'''    
    try:
        df_metrics = spark.table("workspace.default.srag_metrics").toPandas()
        
        data_ref = df_metrics['data_referencia'].iloc[0]
        data_ref = pd.to_datetime(data_ref) 
        
        metricas = {
            "taxa_aumento_casos_7d": f"{df_metrics[df_metrics['metrica']=='taxa_aumento_casos_7d']['valor'].iloc[0]:.2f}%",
            "taxa_mortalidade": f"{df_metrics[df_metrics['metrica']=='taxa_mortalidade']['valor'].iloc[0]:.2f}%",
            "taxa_uti_proxy": f"{df_metrics[df_metrics['metrica']=='taxa_uti_proxy']['valor'].iloc[0]:.2f}%",
            "taxa_vacinacao_proxy": f"{df_metrics[df_metrics['metrica']=='taxa_vacinacao_proxy']['valor'].iloc[0]:.2f}%"
        }
        
        return json.dumps({
            "data_referencia": str(data_ref),
            "metricas": metricas
        }, ensure_ascii=False)
        
    except Exception as e:
        return json.dumps({"erro": str(e)}, ensure_ascii=False)

In [0]:
@tool
def buscar_noticias_srag() -> str:
    '''Busca notícias recentes sobre SRAG no Brasil para embasar o relatório.
    Usa busca real em fontes confiáveis'''
    query = f'SRAG Brasil notícias recentes 2025'

    search = TavilySearch(
        max_results=5,
        search_depth="advanced",
        include_domains=["fiocruz.br", "saude.gov.br", "g1.globo.com", "folha.uol.com.br"]
    )
    
    resultados = search.run(query)
    
    return [thing_found['content'] for thing_found in resultados['results']]
    
    # o de baixo ativar o retorno com fontes

    sources = []
    contents = []

    for r in resultados['results']:
        sources.append(r['url'])
        contents.append(r['content'])
        
    return {
        "sources": sources,
        "contents": contents
    }
                      
tools = [consultar_metricas_srag, buscar_noticias_srag]

In [0]:
import os


buscar_noticias_srag.invoke(None)

In [0]:
consultar_metricas_srag.invoke(None)

In [0]:
analist_prompt_text = '''### Perfil
Você é um Especialista em Inteligência Epidemiológica e Cientista de Dados Sênior. Sua especialidade não é apenas relatar números, mas traduzir dados complexos de SRAG em narrativas analíticas densas, precisas e acionáveis, integrando métricas estatísticas ao contexto socioepidemiológico.

### Objetivo
Produzir uma Análise de Cenário de SRAG em formato de parecer técnico. O output deve ser exclusivamente em **HTML semântico**, pronto para ser injetado em um container relatório web.


### Insumos
- **Base Quantitativa:** {metrics_json}
- **Contexto Qualitativo (Notícias):** {news_json}

### Diretrizes de Escrita e Formatação (Obrigatórias)
1.  **Formato de Saída:** Responda apenas com o código HTML contido dentro de uma tag `<article>`. Não utilize Markdown, não inclua blocos de código ```html, apenas o conteúdo bruto das tags.
2.  **Narrativa Coesa:** Escreva parágrafos analíticos que conectem as métricas de {metrics_json} com os fatos relatados em {news_json}.
3.  **Densidade de Dados:** Cada afirmação deve ser respaldada por valores exatos. Use a tag `<strong>` para destacar indicadores, datas e variações numéricas (ex: "<strong>+24%</strong> na positividade").
4.  **Estilo Visual:** Utilize `<h3>` para títulos de seção. Use a tag `<p style="margin-bottom: 15px; line-height: 1.6;">` para garantir legibilidade entre parágrafos.
5.  **Raciocínio Crítico:** Identifique anomalias. Se os óbitos caíram, mas a internação subiu **15%**, discuta a pressão no sistema.

Não insira título. As métricas e variáveis não são proxy. São originidas do Ministérios da Saúde.

---

### Estrutura do Documento HTML

O agente deve gerar a seguinte estrutura interna:

1.  **<section id="sumario">:** (Sumário Executivo e Diagnóstico) Um parágrafo denso definindo o status epidemiológico, descrevendo a dinâmica da curva e citando a taxa de incidência.
2.  **<section id="vetores">:** (Análise dos Vetores de Propagação) Argumentação que cruza métricas de faixa etária/região com as notícias.
3.  **<section id="pressao">:** (Zonas de Pressão e Alertas Críticos) Foco em métricas de ocupação e picos súbitos, formatado como um alerta analítico.
'''

prompt_template_analist = ChatPromptTemplate.from_messages(
  [
    ('system', analist_prompt_text),
    ('human', ""),
  ]
)

In [0]:
reviewer_prompt_text = '''
Atue como um Revisor de Dados Epidemiológicos e Analista de Saúde Pública. Sua tarefa é revisar uma análise técnica que correlaciona notícias atuais com métricas de SRAG (Síndrome Respiratória Aguda Grave).
    Seus Objetivos Principais:

        Auditoria de Presença: Verifique se todas as métricas fornecidas nos dados brutos foram citadas no texto. Liste as métricas que porventura foram esquecidas.

        Verificação de Consistência: Cheque se os números citados no texto condizem exatamente com os dados brutos (evite alucinações de valores).

        Análise de Correlação: Avalie se a relação estabelecida entre a notícia (ex: "frente fria no Sul") e a métrica (ex: "aumento de internações por Influenza") possui coerência epidemiológica e lógica.

        Rigor Técnico: Aponte se há conclusões precipitadas ou interpretações errôneas sobre tendências (ex: confundir aumento sazonal esperado com uma nova epidemia sem evidências).

    Cite em sua Resposta:

        Métricas Ausentes: (Liste aqui caso o autor tenha ignorado algum dado importante).

        Erros de Fato/Números: (Liste aqui se algum dado foi citado incorretamente).

        Avaliação de Relacionamento Notícia vs. Métrica: (Comente se a interpretação dos fatos sobre as métricas está correta e lógica).

        Veredito Final: (Aprovado / Aprovado com Ressalvas / Necessita Revisão Urgente).

    Dados para Revisão:

    {metrics_json}
    Texto da Análise:

    {analysis}

Dicas para obter o melhor resultado:

    Separação Clara: No campo "Dados para Revisão", organize por tópicos ou em formato de tabela simples.

    Contexto Geográfico: Critique se a notícia usada é nacional ou local.

    Critério de Rigor: Seja extremamente crítico e não aceite generalizações sem embasamento nos dados fornecidos.

    Não faça recomendações. Não coloque título e data. Apenas faça a análise.
    
    '''

prompt_template_reviewer = ChatPromptTemplate.from_messages(
  [
    ('system', reviewer_prompt_text),
    ('human', "Analise os dados e gere uma resposta JSON estrito."),
  ]
)

In [0]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

setup_dados = RunnableParallel(
    {
        "metrics_json": consultar_metricas_srag,
        "news_json": buscar_noticias_srag
    }
)

analista_chain = prompt_template_analist | llm | StrOutputParser()

revisor_chain = prompt_template_reviewer | llm | StrOutputParser()

full_chain = (
    setup_dados 
    | RunnablePassthrough.assign(
        analysis = analista_chain
    )
    | RunnablePassthrough.assign(
        final_review = revisor_chain
    )
)

resultado = full_chain.invoke(None)
print(*[f'{key}: {value}' for key,value in resultado.items()],sep='\n\n\n')

In [0]:
def fig_to_base64(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    return img_base64

df_graph = spark.table("workspace.default.srag_graph").toPandas()

data_ref = df_graph['DT_SIN_PRI'].iloc[0]
data_ref = pd.to_datetime(data_ref) 

# Gráfico 1: Últimos 30 dias (diário)
inicio_30 = data_ref - timedelta(days=29)
df_30 = df_graph[df_graph['DT_SIN_PRI'] >= inicio_30].copy()
casos_diarios = df_30.groupby(df_30['DT_SIN_PRI'].dt.date).size().reset_index(name='Casos')
casos_diarios['DT_SIN_PRI'] = pd.to_datetime(casos_diarios['DT_SIN_PRI'])

fig1 = plt.figure(figsize=(8, 4.5))
plt.plot(casos_diarios['DT_SIN_PRI'], casos_diarios['Casos'], marker='o')
plt.title('Número Diário de Casos de SRAG (Últimos 30 Dias)')
plt.xlabel('Data')
plt.ylabel('Número de Casos')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
graph1_base64 = fig_to_base64(fig1)

# Gráfico 2: Últimos 12 meses (mensal)
inicio_12 = data_ref - timedelta(days=365)
df_12 = df_graph[df_graph['DT_SIN_PRI'] >= inicio_12].copy()
df_12['Mes'] = df_12['DT_SIN_PRI'].dt.to_period('M')
casos_mensais = df_12.groupby('Mes').size().reset_index(name='Casos')
casos_mensais['Mes'] = casos_mensais['Mes'].dt.to_timestamp()

fig2 = plt.figure(figsize=(8, 4.5))
plt.bar(casos_mensais['Mes'], casos_mensais['Casos'], width=20)
plt.title('Número Mensal de Casos de SRAG (Últimos 12 Meses)')
plt.xlabel('Mês')
plt.ylabel('Número de Casos')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
graph2_base64 = fig_to_base64(fig2)

In [0]:
with open('report_template.html', 'r') as f:
    coisa = f.read()

coisa = coisa.replace('{', '{{').replace('}', '}}').replace('{{{{', '{').replace('}}}}', '}')

with open('report_srag_gerado.html', 'w') as f:
    f.write(coisa)

In [0]:
with open('report_template.html', 'r') as f:
    html_text = f.read()

metrics = json.loads(resultado['metrics_json'])

report = html_text.format(
    REPORT_DATE=metrics['data_referencia'],
    CASES=metrics['metricas']['taxa_aumento_casos_7d'],
    MORTALITY_RATE=metrics['metricas']['taxa_mortalidade'],
    UTI_HOSPITALIZATIONS=metrics['metricas']['taxa_uti_proxy'],
    VACCINATION_RATE=metrics['metricas']['taxa_vacinacao_proxy'],
    CHART_1=f'<img src="data:image/png;base64,{graph1_base64}" class="w-full h-full object-contain" alt="Casos diários SRAG" />',
    CHART_2=f'<img src="data:image/png;base64,{graph2_base64}" class="w-full h-full object-contain" alt="Casos mensais SRAG" />',
    MAIN_TEXT=resultado['analysis']
).replace('{{', '{').replace('}}', '}')

# Salva o HTML gerado
with open('report_srag_gerado.html', 'w', encoding='utf-8') as f:
    f.write(report)